In [4]:
with open('input.txt', 'r') as f:
    layout = f.read()
print(layout)

The first floor contains a promethium generator and a promethium-compatible microchip.
The second floor contains a cobalt generator, a curium generator, a ruthenium generator, and a plutonium generator.
The third floor contains a cobalt-compatible microchip, a curium-compatible microchip, a ruthenium-compatible microchip, and a plutonium-compatible microchip.
The fourth floor contains nothing relevant.


In [121]:
def legal_floor(floor):
    "Return True if floor is without MicroChips being fried."    
    for i in range(1, len(floor), 2):
        gen, chip = floor[i-1], floor[i]
        if not gen and chip:
            # No saftey generator
            if any(floor[j] != 0 for j in range(0, len(floor), 2)):
                return False
    return True


def moves(layout, elevator):
    for shift in [-1, 1]:
        level = elevator + shift
        if 0 <= level <= 2:
            floor = layout[level]
            for i in range(len(floor)):
                if layout[elevator][i] != 0:
                    for j in range(i, len(floor)):
                        if layout[elevator][j]:
                            old_i, old_j = layout[level][i], layout[level][j]
                            layout[level][i], layout[level][j] = layout[elevator][i], layout[elevator][j]
                            layout[elevator][i] = layout[elevator][j] = 0
                            if legal_floor(layout[level]) and legal_floor(layout[elevator]):
                                yield shift, i, j
                            layout[elevator][i], layout[elevator][j] = layout[level][i], layout[level][j]
                            layout[level][i], layout[level][j] = old_i, old_j

def str_state(layout, elevator, curr_moves):
    "Unique string identifier for a state. To store searched states, without full matrix in memory."
    s = str(elevator)
    for col in layout.T:
        for i, floor in enumerate(col):
            if floor:
                s += str(i)
    return s + '@' + str(curr_moves)
                            
def backtrack(layout, elevator, curr_moves, cutoff, searched, path):
    state = str_state(layout, elevator, curr_moves)
    if curr_moves >= cutoff or state in searched:
        return cutoff
    if state.startswith('0'*(layout.shape[1]+1)):
        return min(cutoff, curr_moves)
        
    searched.add(state)
        
    path.append([])
    for shift, i, j in moves(layout, elevator):
        # Search move
        new_cutoff = backtrack(layout, elevator+shift, curr_moves+1, cutoff, searched, path)
        if new_cutoff < cutoff:
            cutoff = new_cutoff
            path[curr_moves] = (shift, i, j)
    return cutoff
                

def iterative_deepening(layout, elevator, max_depth=50):
    cutoff = 0
    while cutoff < max_depth:
        print('At depth', cutoff)
        cutoff += 1
        path = [[]]
        searched = set()
        best = backtrack(layout, elevator, 0, cutoff, searched, path)
        if best < cutoff:
            return best, path[:best]

In [110]:
layout = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
                   [0, 0, 1, 0, 1, 0, 1, 0, 1, 0],
                   [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])
elevator = 3

searched = set()
path = [[]]
best = backtrack(layout, elevator, 0, 50, searched, path)

#best, path = iterative_deepening(layout, elevator)
print(best)
print(path)

KeyboardInterrupt: 

In [124]:
layout = np.array([[0,0,0,0,0,0,0,0,0,0],
                   [1,1,1,1,1,1,1,1,1,1],
                   [0,0,0,0,0,0,0,0,0,0]])
best, path = iterative_deepening(layout, 1, max_depth=100)
print(best, path)

At depth 0
At depth 1
At depth 2
At depth 3
At depth 4
At depth 5
At depth 6
At depth 7
At depth 8
At depth 9
At depth 10
At depth 11
At depth 12
At depth 13
At depth 14
At depth 15
At depth 16
At depth 17
At depth 18
At depth 19
At depth 20
At depth 21
At depth 22
At depth 23
At depth 24
At depth 25
25 [(-1, 0, 1), (1, 0, 0), (1, 2, 3), (-1, 2, 2), (-1, 0, 2), (1, 2, 2), (1, 5, 7), (-1, 3, 3), (-1, 4, 6), (1, 4, 4), (-1, 2, 3), (1, 6, 6), (-1, 4, 6), (1, 4, 4), (-1, 4, 8), (1, 1, 1), (-1, 1, 9), (1, 1, 1), (1, 1, 1), (-1, 1, 5), (-1, 1, 5), (1, 1, 1), (1, 1, 1), (-1, 1, 7), (-1, 1, 7)]
